In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import os
from tqdm import tqdm
import sys

# 한글 출력 인코딩 설정 (Windows)
if sys.platform == 'win32':
    import io
    sys.stdout = io.TextIOWrapper(sys.stdout.buffer, encoding='utf-8')
    sys.stderr = io.TextIOWrapper(sys.stderr.buffer, encoding='utf-8')

# 설정
DATA_FILE = 'blackjack_probabilities_106cols_traindata.csv'
MODEL_SAVE_PATH = 'blackjack_model.pth'
BATCH_SIZE = 128
LEARNING_RATE = 0.001
NUM_EPOCHS = 100
TEST_SIZE = 0.2
RANDOM_SEED = 42

# CUDA 사용 가능 여부 확인
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")


In [ ]:
class BlackjackDataset(Dataset):
    """블랙잭 데이터셋 클래스"""
    def __init__(self, features, labels):
        self.features = torch.FloatTensor(features)
        self.labels = torch.FloatTensor(labels)
    
    def __len__(self):
        return len(self.features)
    
    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]


In [ ]:
# 여러 모델 변형 정의

# 1. 기본 모델 (ReLU 사용)
class BlackjackModelBase(nn.Module):
    """기본 모델: 104 -> 256 -> 128 -> 64 -> 2 (ReLU)"""
    def __init__(self, input_size=104, dropout=0.3):
        super().__init__()
        self.fc1 = nn.Linear(input_size, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 2)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.relu(self.fc3(x))
        x = self.sigmoid(self.fc4(x))
        return x


# 2. 전부 Sigmoid 버전
class BlackjackModelSigmoid(nn.Module):
    """Sigmoid 모델: 104 -> 256 -> 128 -> 64 -> 2 (전부 Sigmoid)"""
    def __init__(self, input_size=104, dropout=0.3):
        super().__init__()
        self.fc1 = nn.Linear(input_size, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 2)
        self.sigmoid = nn.Sigmoid()
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):
        x = self.sigmoid(self.fc1(x))
        x = self.dropout(x)
        x = self.sigmoid(self.fc2(x))
        x = self.dropout(x)
        x = self.sigmoid(self.fc3(x))
        x = self.sigmoid(self.fc4(x))
        return x


# 3. 더 깊은 구조
class BlackjackModelDeep(nn.Module):
    """깊은 모델: 104 -> 512 -> 256 -> 128 -> 64 -> 32 -> 2"""
    def __init__(self, input_size=104, dropout=0.3):
        super().__init__()
        self.fc1 = nn.Linear(input_size, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, 64)
        self.fc5 = nn.Linear(64, 32)
        self.fc6 = nn.Linear(32, 2)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.relu(self.fc3(x))
        x = self.dropout(x)
        x = self.relu(self.fc4(x))
        x = self.dropout(x)
        x = self.relu(self.fc5(x))
        x = self.sigmoid(self.fc6(x))
        return x


# 4. 더 얕은 구조
class BlackjackModelShallow(nn.Module):
    """얕은 모델: 104 -> 128 -> 64 -> 2"""
    def __init__(self, input_size=104, dropout=0.3):
        super().__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 2)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.sigmoid(self.fc3(x))
        return x


# 5. 더 넓은 구조
class BlackjackModelWide(nn.Module):
    """넓은 모델: 104 -> 512 -> 512 -> 2"""
    def __init__(self, input_size=104, dropout=0.3):
        super().__init__()
        self.fc1 = nn.Linear(input_size, 512)
        self.fc2 = nn.Linear(512, 512)
        self.fc3 = nn.Linear(512, 2)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.sigmoid(self.fc3(x))
        return x


# 6. 다른 구조
class BlackjackModelAlt(nn.Module):
    """대안 모델: 104 -> 256 -> 256 -> 128 -> 2"""
    def __init__(self, input_size=104, dropout=0.3):
        super().__init__()
        self.fc1 = nn.Linear(input_size, 256)
        self.fc2 = nn.Linear(256, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, 2)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.relu(self.fc3(x))
        x = self.sigmoid(self.fc4(x))
        return x


# 모델 정의 딕셔너리
MODEL_CONFIGS = {
    'base': {'class': BlackjackModelBase, 'name': '기본 모델 (ReLU)'},
    'sigmoid': {'class': BlackjackModelSigmoid, 'name': '전부 Sigmoid'},
    'deep': {'class': BlackjackModelDeep, 'name': '깊은 구조'},
    'shallow': {'class': BlackjackModelShallow, 'name': '얕은 구조'},
    'wide': {'class': BlackjackModelWide, 'name': '넓은 구조'},
    'alt': {'class': BlackjackModelAlt, 'name': '대안 구조'}
}

# 호환성을 위해 기본 모델을 BlackjackModel로도 사용 가능
BlackjackModel = BlackjackModelBase


In [ ]:
def load_and_preprocess_data(file_path):
    """
    CSV 파일을 로드하고 전처리
    같은 입력 조합에 대해 시뮬레이션 결과의 평균을 계산하여 확률로 변환
    """
    print(f"Loading data from {file_path}...")
    df = pd.read_csv(file_path)
    
    # 입력 특성 컬럼 추출 (rem_*와 curr_*)
    input_cols = [col for col in df.columns if col.startswith('rem_') or col.startswith('curr_')]
    
    # 출력 컬럼
    output_cols = ['player_burst', 'dealer_wins']
    
    print(f"Found {len(df)} rows")
    print(f"Input features: {len(input_cols)}")
    
    # 같은 입력 조합에 대해 그룹화하여 확률 계산
    print("Grouping by input features and calculating probabilities...")
    grouped = df.groupby(input_cols)[output_cols].mean().reset_index()
    
    # 확률값으로 변환 (이미 평균이므로 0~1 사이의 확률값)
    X = grouped[input_cols].values
    y = grouped[output_cols].values
    
    # 컬럼명 변경: player_burst -> hit_bust_prob, dealer_wins -> stay_dealer_win_prob
    y_df = pd.DataFrame(y, columns=['hit_bust_prob', 'stay_dealer_win_prob'])
    y = y_df.values
    
    print(f"After grouping: {len(X)} unique scenarios")
    print(f"Output shape: {y.shape}")
    print(f"hit_bust_prob range: [{y[:, 0].min():.4f}, {y[:, 0].max():.4f}]")
    print(f"stay_dealer_win_prob range: [{y[:, 1].min():.4f}, {y[:, 1].max():.4f}]")
    
    return X, y, input_cols


In [ ]:
def train_model(model, train_loader, val_loader, num_epochs, learning_rate):
    """모델 학습"""
    criterion = nn.MSELoss()  # 확률 예측이므로 MSE 사용
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    train_losses = []
    val_losses = []
    
    best_val_loss = float('inf')
    best_model_state = None
    
    for epoch in range(num_epochs):
        # 학습 모드
        model.train()
        train_loss = 0.0
        
        # tqdm 진행 표시줄 생성 (postfix로 loss 표시 예정)
        pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")
        
        for features, labels in pbar:
            features = features.to(device)
            labels = labels.to(device)
            
            # Forward pass
            outputs = model(features)
            loss = criterion(outputs, labels)
            
            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
        
        # 검증 모드
        model.eval()
        val_loss = 0.0
        
        with torch.no_grad():
            for features, labels in val_loader:
                features = features.to(device)
                labels = labels.to(device)
                
                outputs = model(features)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
        
        avg_train_loss = train_loss / len(train_loader)
        avg_val_loss = val_loss / len(val_loader)
        
        train_losses.append(avg_train_loss)
        val_losses.append(avg_val_loss)
        
        # 최고 성능 모델 저장
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            best_model_state = model.state_dict().copy()
        
        # tqdm 진행 표시줄에 loss 정보 업데이트
        pbar.set_postfix({
            'Train Loss': f'{avg_train_loss:.6f}',
            'Val Loss': f'{avg_val_loss:.6f}'
        })
        pbar.close()
    
    # 최고 성능 모델로 복원
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
    
    return train_losses, val_losses, best_val_loss


In [ ]:
def evaluate_model(model, test_loader):
    """모델 평가"""
    model.eval()
    criterion = nn.MSELoss()
    
    total_loss = 0.0
    
    with torch.no_grad():
        for features, labels in test_loader:
            features = features.to(device)
            labels = labels.to(device)
            
            outputs = model(features)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
    
    avg_loss = total_loss / len(test_loader)
    return avg_loss


In [ ]:
# 데이터 로드 및 전처리
if not os.path.exists(DATA_FILE):
    print(f"Error: Data file '{DATA_FILE}' not found!")
    print("Please run create_training_data.py first to generate the training data.")
else:
    X, y, input_cols = load_and_preprocess_data(DATA_FILE)
    
    # 데이터 정규화
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Train/Validation/Test 분할
    X_temp, X_test, y_temp, y_test = train_test_split(
        X_scaled, y, test_size=TEST_SIZE, random_state=RANDOM_SEED
    )
    X_train, X_val, y_train, y_val = train_test_split(
        X_temp, y_temp, test_size=TEST_SIZE, random_state=RANDOM_SEED
    )
    
    print(f"\nData split:")
    print(f"  Train: {len(X_train)} samples")
    print(f"  Validation: {len(X_val)} samples")
    print(f"  Test: {len(X_test)} samples")


In [ ]:
# 데이터셋 및 데이터로더 생성
train_dataset = BlackjackDataset(X_train, y_train)
val_dataset = BlackjackDataset(X_val, y_val)
test_dataset = BlackjackDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)


In [ ]:
# 모델 생성
model = BlackjackModel(input_size=len(input_cols)).to(device)
print(f"\nModel architecture:")
print(model)

# 모델 파라미터 수 계산
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"\nTotal parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")


In [ ]:
# 모든 모델 학습 및 저장
print("=" * 70)
print("모든 모델 학습 시작")
print("=" * 70)

trained_models = {}
model_results = {}

for model_key, config in MODEL_CONFIGS.items():
    print(f"\n{'='*70}")
    print(f"학습 중: {config['name']}")
    print(f"{'='*70}")
    
    # 모델 생성
    model = config['class'](input_size=len(input_cols)).to(device)
    
    # 파라미터 수 계산
    total_params = sum(p.numel() for p in model.parameters())
    print(f"파라미터 수: {total_params:,}")
    
    # 학습
    train_losses, val_losses, best_val_loss = train_model(
        model, train_loader, val_loader, NUM_EPOCHS, LEARNING_RATE
    )
    
    # 테스트 평가
    test_loss = evaluate_model(model, test_loader)
    
    # 모델 저장
    model_path = f'blackjack_model_{model_key}.pth'
    torch.save({
        'model_state_dict': model.state_dict(),
        'model_config': {
            'model_type': model_key,
            'input_size': len(input_cols),
            'class_name': config['class'].__name__
        },
        'scaler': scaler,
        'best_val_loss': best_val_loss,
        'test_loss': test_loss
    }, model_path)
    
    trained_models[model_key] = model
    model_results[model_key] = {
        'name': config['name'],
        'best_val_loss': best_val_loss,
        'test_loss': test_loss,
        'params': total_params,
        'model_path': model_path
    }
    
    print(f"\n{config['name']} 학습 완료!")
    print(f"  Best Val Loss: {best_val_loss:.6f}")
    print(f"  Test Loss: {test_loss:.6f}")
    print(f"  모델 저장: {model_path}")

print(f"\n{'='*70}")
print("모든 모델 학습 완료!")
print(f"{'='*70}")


In [ ]:
# 모델 성능 비교
print("\n" + "=" * 70)
print("모델 성능 비교")
print("=" * 70)
print(f"{'모델명':<20} {'파라미터 수':<15} {'Val Loss':<15} {'Test Loss':<15}")
print("-" * 70)

sorted_results = sorted(model_results.items(), key=lambda x: x[1]['test_loss'])

for model_key, result in sorted_results:
    print(f"{result['name']:<20} {result['params']:>14,} {result['best_val_loss']:>14.6f} {result['test_loss']:>14.6f}")

best_model_key = sorted_results[0][0]
print(f"\n최고 성능 모델: {model_results[best_model_key]['name']}")
print(f"Test Loss: {model_results[best_model_key]['test_loss']:.6f}")


In [ ]:
# 모델 학습
print(f"\nStarting training...")
print(f"  Epochs: {NUM_EPOCHS}")
print(f"  Batch size: {BATCH_SIZE}")
print(f"  Learning rate: {LEARNING_RATE}")
print()

train_losses, val_losses, best_val_loss = train_model(model, train_loader, val_loader, NUM_EPOCHS, LEARNING_RATE)


In [ ]:
# 테스트 평가
print("\nEvaluating on test set...")
test_loss = evaluate_model(model, test_loader)
print(f"Test Loss: {test_loss:.6f}")


In [ ]:
# 모델 저장
print(f"\nSaving model to {MODEL_SAVE_PATH}...")
torch.save({
    'model_state_dict': model.state_dict(),
    'input_cols': input_cols,
    'scaler': scaler,
    'model_config': {
        'input_size': len(input_cols),
        'hidden1': 256,
        'hidden2': 128,
        'hidden3': 64,
        'output_size': 2,
        'dropout': 0.3
    }
}, MODEL_SAVE_PATH)

print(f"Model saved successfully!")
print(f"\nTraining completed!")


In [ ]:
# 모든 모델로 게임 시뮬레이션 실행 및 최종 비교
print("\n" + "=" * 70)
print("모든 모델로 게임 시뮬레이션 실행")
print("=" * 70)

game_results = {}
n_games = 100

for model_key, config in MODEL_CONFIGS.items():
    print(f"\n{config['name']} 시뮬레이션 중...")
    
    # 모델 로드
    model_path = f'blackjack_model_{model_key}.pth'
    if os.path.exists(model_path):
        checkpoint = torch.load(model_path, map_location=device)
        
        # 모델 생성
        model_class = config['class']
        model = model_class(input_size=len(input_cols)).to(device)
        model.load_state_dict(checkpoint['model_state_dict'])
        model.eval()
        
        scaler = checkpoint.get('scaler', None)
        
        # 게임 시뮬레이션
        results = play_blackjack_game(model, scaler, n_games=n_games)
        
        game_results[model_key] = {
            'name': config['name'],
            **results
        }
        
        print(f"  완료! 총 점수: {results['total_score']}점, 승률: {results['win_rate']:.1%}")
    else:
        print(f"  모델 파일을 찾을 수 없습니다: {model_path}")

# 최종 결과 비교
print("\n" + "=" * 70)
print("최종 게임 시뮬레이션 결과 비교")
print("=" * 70)
print(f"{'모델명':<20} {'총 점수':<12} {'평균 점수':<12} {'승률':<10} {'승리':<8} {'패배':<8} {'무승부':<8}")
print("-" * 90)

sorted_game_results = sorted(game_results.items(), key=lambda x: x[1]['total_score'], reverse=True)

for model_key, result in sorted_game_results:
    print(f"{result['name']:<20} {result['total_score']:>10}점 {result['average_score']:>10.2f}점 {result['win_rate']:>8.1%} {result['wins']:>6}회 {result['losses']:>6}회 {result['ties']:>6}회")

best_game_model = sorted_game_results[0][0]
print(f"\n최고 성능 모델 (게임 시뮬레이션): {game_results[best_game_model]['name']}")
print(f"총 점수: {game_results[best_game_model]['total_score']}점")
print(f"평균 점수: {game_results[best_game_model]['average_score']:.2f}점/게임")
print(f"승률: {game_results[best_game_model]['win_rate']:.1%}")
print(f"예상 수익: {game_results[best_game_model]['total_score'] - (n_games * 5)}점 (판돈 제외)")


## CSV 데이터로 모델 Validation

`blackjack_probabilities_data.csv` 파일의 실제 확률값과 모델 예측값을 비교하여 검증합니다.


In [ ]:
# CSV 데이터로 모델 Validation

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# 한글 폰트 설정
plt.rcParams['font.family'] = 'Malgun Gothic'  # Windows
plt.rcParams['axes.unicode_minus'] = False  # 마이너스 기호 깨짐 방지

# 한글 폰트가 없을 경우를 대비한 폴백
try:
    # Windows에서 사용 가능한 한글 폰트 확인
    font_list = [f.name for f in fm.fontManager.ttflist]
    if 'Malgun Gothic' not in font_list:
        # 다른 한글 폰트 시도
        for font in ['NanumGothic', 'NanumBarunGothic', 'AppleGothic', 'Gulim']:
            if font in font_list:
                plt.rcParams['font.family'] = font
                break
except:
    pass

def validate_model_with_csv(model, scaler, csv_path='blackjack_probabilities_data.csv'):
    """
    CSV 파일의 실제 확률값과 모델 예측값을 비교하여 검증
    
    Args:
        model: 학습된 모델
        scaler: 데이터 정규화용 scaler
        csv_path: 검증용 CSV 파일 경로
    
    Returns:
        dict: 검증 결과 메트릭
    """
    # CSV 파일 읽기
    if not os.path.exists(csv_path):
        print(f"오류: {csv_path} 파일을 찾을 수 없습니다.")
        return None
    
    df = pd.read_csv(csv_path)
    print(f"CSV 파일 로드 완료: {len(df)}개 샘플")
    
    # 입력/출력 분리
    input_cols = [col for col in df.columns if col.startswith('rem_') or col.startswith('curr_')]
    output_cols = ['hit_bust_prob', 'stay_dealer_win_prob']
    
    X_csv = df[input_cols].values
    y_true = df[output_cols].values
    
    print(f"입력 차원: {X_csv.shape}")
    print(f"출력 차원: {y_true.shape}")
    
    # 데이터 정규화
    if scaler is not None:
        X_scaled = scaler.transform(X_csv)
    else:
        X_scaled = X_csv
    
    # 모델 예측
    model.eval()
    predictions = []
    
    with torch.no_grad():
        # 배치로 처리
        batch_size = 128
        for i in range(0, len(X_scaled), batch_size):
            batch = X_scaled[i:i+batch_size]
            batch_tensor = torch.FloatTensor(batch).to(device)
            batch_pred = model(batch_tensor)
            predictions.append(batch_pred.cpu().numpy())
    
    y_pred = np.concatenate(predictions, axis=0)
    
    # 메트릭 계산
    mse_hit = mean_squared_error(y_true[:, 0], y_pred[:, 0])
    mse_dealer = mean_squared_error(y_true[:, 1], y_pred[:, 1])
    mse_total = mean_squared_error(y_true, y_pred)
    
    mae_hit = mean_absolute_error(y_true[:, 0], y_pred[:, 0])
    mae_dealer = mean_absolute_error(y_true[:, 1], y_pred[:, 1])
    mae_total = mean_absolute_error(y_true, y_pred)
    
    r2_hit = r2_score(y_true[:, 0], y_pred[:, 0])
    r2_dealer = r2_score(y_true[:, 1], y_pred[:, 1])
    r2_total = r2_score(y_true, y_pred)
    
    # RMSE 계산
    rmse_hit = np.sqrt(mse_hit)
    rmse_dealer = np.sqrt(mse_dealer)
    rmse_total = np.sqrt(mse_total)
    
    results = {
        'mse': {
            'hit_bust_prob': mse_hit,
            'stay_dealer_win_prob': mse_dealer,
            'total': mse_total
        },
        'mae': {
            'hit_bust_prob': mae_hit,
            'stay_dealer_win_prob': mae_dealer,
            'total': mae_total
        },
        'rmse': {
            'hit_bust_prob': rmse_hit,
            'stay_dealer_win_prob': rmse_dealer,
            'total': rmse_total
        },
        'r2': {
            'hit_bust_prob': r2_hit,
            'stay_dealer_win_prob': r2_dealer,
            'total': r2_total
        },
        'y_true': y_true,
        'y_pred': y_pred
    }
    
    return results


def print_validation_results(results, model_name="모델"):
    """검증 결과를 보기 좋게 출력"""
    print("\n" + "=" * 70)
    print(f"{model_name} Validation Results")
    print("=" * 70)
    
    print("\n[Mean Squared Error (MSE)]")
    print(f"  hit_bust_prob:        {results['mse']['hit_bust_prob']:.6f}")
    print(f"  stay_dealer_win_prob: {results['mse']['stay_dealer_win_prob']:.6f}")
    print(f"  Total:                {results['mse']['total']:.6f}")
    
    print("\n[Root Mean Squared Error (RMSE)]")
    print(f"  hit_bust_prob:        {results['rmse']['hit_bust_prob']:.6f}")
    print(f"  stay_dealer_win_prob: {results['rmse']['stay_dealer_win_prob']:.6f}")
    print(f"  Total:                {results['rmse']['total']:.6f}")
    
    print("\n[Mean Absolute Error (MAE)]")
    print(f"  hit_bust_prob:        {results['mae']['hit_bust_prob']:.6f}")
    print(f"  stay_dealer_win_prob: {results['mae']['stay_dealer_win_prob']:.6f}")
    print(f"  Total:                {results['mae']['total']:.6f}")
    
    print("\n[R² Score (Coefficient of Determination)]")
    print(f"  hit_bust_prob:        {results['r2']['hit_bust_prob']:.6f}")
    print(f"  stay_dealer_win_prob: {results['r2']['stay_dealer_win_prob']:.6f}")
    print(f"  Total:                {results['r2']['total']:.6f}")
    
    # R² 해석
    print("\n[R² 해석]")
    r2_hit = results['r2']['hit_bust_prob']
    r2_dealer = results['r2']['stay_dealer_win_prob']
    
    if r2_hit >= 0.9:
        hit_quality = "매우 우수"
    elif r2_hit >= 0.7:
        hit_quality = "우수"
    elif r2_hit >= 0.5:
        hit_quality = "보통"
    else:
        hit_quality = "개선 필요"
    
    if r2_dealer >= 0.9:
        dealer_quality = "매우 우수"
    elif r2_dealer >= 0.7:
        dealer_quality = "우수"
    elif r2_dealer >= 0.5:
        dealer_quality = "보통"
    else:
        dealer_quality = "개선 필요"
    
    print(f"  hit_bust_prob:        {hit_quality} (R² = {r2_hit:.4f})")
    print(f"  stay_dealer_win_prob: {dealer_quality} (R² = {r2_dealer:.4f})")


def plot_validation_comparison(results, model_name="모델"):
    """실제값 vs 예측값 비교 그래프"""
    y_true = results['y_true']
    y_pred = results['y_pred']
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # hit_bust_prob
    axes[0].scatter(y_true[:, 0], y_pred[:, 0], alpha=0.5, s=20)
    axes[0].plot([y_true[:, 0].min(), y_true[:, 0].max()], 
                 [y_true[:, 0].min(), y_true[:, 0].max()], 
                 'r--', lw=2, label='Perfect Prediction')
    axes[0].set_xlabel('실제 hit_bust_prob')
    axes[0].set_ylabel('예측 hit_bust_prob')
    axes[0].set_title(f'hit_bust_prob (R² = {results["r2"]["hit_bust_prob"]:.4f})')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # stay_dealer_win_prob
    axes[1].scatter(y_true[:, 1], y_pred[:, 1], alpha=0.5, s=20, color='orange')
    axes[1].plot([y_true[:, 1].min(), y_true[:, 1].max()], 
                 [y_true[:, 1].min(), y_true[:, 1].max()], 
                 'r--', lw=2, label='Perfect Prediction')
    axes[1].set_xlabel('실제 stay_dealer_win_prob')
    axes[1].set_ylabel('예측 stay_dealer_win_prob')
    axes[1].set_title(f'stay_dealer_win_prob (R² = {results["r2"]["stay_dealer_win_prob"]:.4f})')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.suptitle(f'{model_name} - 실제값 vs 예측값 비교', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()


print("Validation 함수 정의 완료!")


In [ ]:
# 모든 모델에 대해 CSV 데이터로 Validation 수행

print("=" * 70)
print("CSV 데이터로 모델 Validation")
print("=" * 70)

validation_results = {}

for model_key, config in MODEL_CONFIGS.items():
    model_path = f'blackjack_model_{model_key}.pth'
    
    if os.path.exists(model_path):
        print(f"\n{'='*70}")
        print(f"검증 중: {config['name']}")
        print(f"{'='*70}")
        
        # 모델 로드
        checkpoint = torch.load(model_path, map_location=device)
        model_config = checkpoint['model_config']
        scaler = checkpoint.get('scaler', None)
        
        # 모델 생성
        model_type = model_config.get('model_type', model_key)
        if model_type in MODEL_CONFIGS:
            model_class = MODEL_CONFIGS[model_type]['class']
            model = model_class(input_size=model_config['input_size']).to(device)
        else:
            model = BlackjackModelBase(input_size=model_config['input_size']).to(device)
        
        model.load_state_dict(checkpoint['model_state_dict'])
        model.eval()
        
        # Validation 수행
        results = validate_model_with_csv(model, scaler, csv_path='blackjack_probabilities_data.csv')
        
        if results:
            validation_results[model_key] = {
                'name': config['name'],
                'results': results
            }
            
            # 결과 출력
            print_validation_results(results, model_name=config['name'])
            
            # 그래프 출력
            plot_validation_comparison(results, model_name=config['name'])
    else:
        print(f"\n{config['name']}: 모델 파일을 찾을 수 없습니다 ({model_path})")

print("\n" + "=" * 70)
print("모든 모델 Validation 완료")
print("=" * 70)


In [ ]:
# Validation 결과 비교 요약

if validation_results:
    print("\n" + "=" * 70)
    print("모델별 Validation 결과 비교")
    print("=" * 70)
    
    print(f"\n{'모델명':<20} {'MSE (Total)':<15} {'MAE (Total)':<15} {'RMSE (Total)':<15} {'R² (Total)':<12}")
    print("-" * 80)
    
    for model_key, data in validation_results.items():
        results = data['results']
        print(f"{data['name']:<20} "
              f"{results['mse']['total']:<15.6f} "
              f"{results['mae']['total']:<15.6f} "
              f"{results['rmse']['total']:<15.6f} "
              f"{results['r2']['total']:<12.6f}")
    
    # 최고 성능 모델 찾기
    print("\n" + "=" * 70)
    print("최고 성능 모델 (R² 기준)")
    print("=" * 70)
    
    best_r2_model = max(validation_results.items(), 
                       key=lambda x: x[1]['results']['r2']['total'])
    best_mae_model = min(validation_results.items(), 
                        key=lambda x: x[1]['results']['mae']['total'])
    best_rmse_model = min(validation_results.items(), 
                         key=lambda x: x[1]['results']['rmse']['total'])
    
    print(f"\n최고 R²: {best_r2_model[1]['name']} (R² = {best_r2_model[1]['results']['r2']['total']:.6f})")
    print(f"최저 MAE: {best_mae_model[1]['name']} (MAE = {best_mae_model[1]['results']['mae']['total']:.6f})")
    print(f"최저 RMSE: {best_rmse_model[1]['name']} (RMSE = {best_rmse_model[1]['results']['rmse']['total']:.6f})")
    
    # 상세 비교
    print("\n" + "=" * 70)
    print("hit_bust_prob 예측 성능 비교")
    print("=" * 70)
    print(f"{'모델명':<20} {'MSE':<15} {'MAE':<15} {'RMSE':<15} {'R²':<12}")
    print("-" * 80)
    
    sorted_hit = sorted(validation_results.items(), 
                       key=lambda x: x[1]['results']['r2']['hit_bust_prob'], 
                       reverse=True)
    
    for model_key, data in sorted_hit:
        results = data['results']
        print(f"{data['name']:<20} "
              f"{results['mse']['hit_bust_prob']:<15.6f} "
              f"{results['mae']['hit_bust_prob']:<15.6f} "
              f"{results['rmse']['hit_bust_prob']:<15.6f} "
              f"{results['r2']['hit_bust_prob']:<12.6f}")
    
    print("\n" + "=" * 70)
    print("stay_dealer_win_prob 예측 성능 비교")
    print("=" * 70)
    print(f"{'모델명':<20} {'MSE':<15} {'MAE':<15} {'RMSE':<15} {'R²':<12}")
    print("-" * 80)
    
    sorted_dealer = sorted(validation_results.items(), 
                          key=lambda x: x[1]['results']['r2']['stay_dealer_win_prob'], 
                          reverse=True)
    
    for model_key, data in sorted_dealer:
        results = data['results']
        print(f"{data['name']:<20} "
              f"{results['mse']['stay_dealer_win_prob']:<15.6f} "
              f"{results['mae']['stay_dealer_win_prob']:<15.6f} "
              f"{results['rmse']['stay_dealer_win_prob']:<15.6f} "
              f"{results['r2']['stay_dealer_win_prob']:<12.6f}")
else:
    print("검증 결과가 없습니다. 먼저 모델을 학습시켜야 합니다.")
